In [1]:
%%writefile requirements.txt
tensorflow==2.3.1
nltk==3.5
colorama==0.4.3
numpy==1.18.5
scikit_learn==0.23.2
Flask==1.1.2

Writing requirements.txt


In [14]:
!pip install colorama==0.4.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [3]:
%%writefile intents.json

{"intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?" ],
     "responses": ["I.m Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["what is your name", "what should I call you", "whats your name?"],
    "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
    "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "createaccount",
    "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
    "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
    },
    {"tag": "complaint",
    "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
    "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
    }
]
}

Writing intents.json


In [4]:
with open('intents.json') as file:
    data = json.load(file)

In [5]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [6]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [7]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = ""

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [8]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [10]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
2/2 [==============================] - 1s 19ms/step - loss: 2.0803 - accuracy: 0.0909
Epoch 2/550
2/2 [==============================] - 0s 5ms/step - loss: 2.0786 - accuracy: 0.2121
Epoch 3/550
2/2 [==============================] - 0s 4ms/step - loss: 2.0780 - accuracy: 0.2121
Epoch 4/550
2/2 [==============================] - 0s 4ms/step - loss: 2.0779 - accuracy: 0.1818
Epoch 5/550
2/2 [==============================] - 0s 4ms/step - loss: 2.0779 - accuracy: 0.1515
Epoch 6/550
2/2 [==============================] - 0s 4ms/step - loss: 2.0776 - accuracy: 0.1515
Epoch 7/550
2/2 [==============================] - 0s 2ms/step - loss: 2.0773 - accuracy: 0.1515
Epoch 8/550
2/2 [==============================] - 0s 7ms/step - loss: 2.0768 - accuracy: 0.1515
Epoch 9/550
2/2 [==============================] - 0s 4ms/step - loss: 2.0761 - accuracy: 0.2121
Epoch 10/550
2/2 [==============================] - 0s 6ms/step - loss: 2.0753 - accuracy: 0.3333
Epoch 11/550
2/2 [==========

In [11]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

# It's the time to test our chat-model now

In [12]:
%%writefile chat.py

import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

Writing chat.py


In [20]:
!python chat.py

Start messaging with the bot (type quit to stop)!
2022-11-08 18:37:45.025210: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
User: Hello
1/1 [==============================] - 0s 64ms/step
ChatBot: Hi there
User: who are you
1/1 [==============================] - 0s 11ms/step
ChatBot: I.m Joana, your bot assistant
User: that's nice
1/1 [==============================] - 0s 12ms/step
ChatBot: Hi there
User: could you help me
1/1 [==============================] - 0s 12ms/step
ChatBot: Yes Sure, How can I support you
User: create account for me
1/1 [==============================] - 0s 12ms/step
ChatBot: You can just easily create a new account from our web site
User: okay
1/1 [==============================] - 0s 13ms/step
ChatBot: Hi
User: thank you
1/1 [==============================] - 0s 11ms/step
ChatBot: Happy to help!
User: thanks a lot
1/1 [==============================] - 0s 13ms/step
ChatBo